In [286]:
import pandas as pd
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("amzn-products.db")
df = pd.read_sql_query("SELECT * from product ", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head(5))

con.close()

          ASIN                                       Product_Name  \
0  B08FDG42VZ   Dreamiracle Ice Maker Machine for Countertop  ...   
1  B08636YHJX   IKTCH 30-inch Wall Mount Range Hood 900 CFM Du...   
2  B07WS78DVK   IKTCH 30 inch Built-in/Insert Range Hood 900 C...   
3  B075XQRXCF   Portable Washing Machine TG23 - Twin Tub Washe...   
4  B08DTGGBW1   Electric Cooktop  ECOTOUCH 12 Radiant Electric...   

           Brand     Category Est_Monthly_Revenue Est_Monthly_Sales     Price  \
0   Dreamiracle   Appliances       $1,173,677.31             6,669   $175.99    
1         IKTCH   Appliances         $202,050.00               450   $449.00    
2         IKTCH   Appliances         $162,991.00               419   $389.00    
3  Think Gizmos   Appliances          $87,378.15               437   $199.95    
4      ECOTOUCH   Appliances          $77,175.46               454   $169.99    

       Fees       Net Rank  Reviews  LQS  Sellers Date_First_Available  \
0   $44.31   $131.68    

### Data Cleaning
#### Trim spaces in all columns in the dataframe

In [287]:
# Trim spaces in all columns in the dataframe

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)


# simple example of trimming whitespace from data elements
df = trim_all_columns(df)
print(df)

           ASIN                                       Product_Name  \
0    B08FDG42VZ  Dreamiracle Ice Maker Machine for Countertop  ...   
1    B08636YHJX  IKTCH 30-inch Wall Mount Range Hood 900 CFM Du...   
2    B07WS78DVK  IKTCH 30 inch Built-in/Insert Range Hood 900 C...   
3    B075XQRXCF  Portable Washing Machine TG23 - Twin Tub Washe...   
4    B08DTGGBW1  Electric Cooktop  ECOTOUCH 12 Radiant Electric...   
..          ...                                                ...   
201  B06XNN9CN7  Williamsburg Handmade Oil Colors Basic Paintin...   
202  B088Y88MF7            da Vinci Brushes 5360 Artist Brush  Set   
203  B08KQCX2NJ  Ambesonne Floral Fabric by The Yard  Vintage G...   
204  B01J1WR9BA  Master Airbrush 1/5 HP Cool Runner II Dual Fan...   
205  B073XSMZCM  6 Gauge  935 Argentium Silver Wire  Half Round...   

                Brand               Category Est_Monthly_Revenue  \
0         Dreamiracle             Appliances       $1,173,677.31   
1               IKTCH  

#### Fixing Currency Columns

In [288]:
# Fixing Currency Columns 

# Est_Monthly_Revenue
df.Est_Monthly_Revenue = [x.strip('$') for x in df.Est_Monthly_Revenue]
df.Est_Monthly_Revenue = df.Est_Monthly_Revenue.str.replace(',', '').astype(float)

# Est_Monthly_Sales
df.Est_Monthly_Sales = df.Est_Monthly_Sales.str.replace(',', '').astype(float)

# Price
df.Price = [x.strip('$') for x in df.Price]
df.Price = df.Price.str.replace(',', '').astype(float)

# Fees
df.Fees = [x.strip('$') for x in df.Fees]
df.Fees = df.Fees.str.replace(',', '').astype(float)

# Net
df.Net = [x.strip('$') for x in df.Net]
df.Net = df.Net.str.replace(',', '').astype(float)


### Fixing Rank column

In [291]:
# Check Data Type in Rank
# See different data types in the same column.  

df['Rank'].apply(type)

0      <class 'int'>
1      <class 'int'>
2      <class 'int'>
3      <class 'int'>
4      <class 'int'>
           ...      
201    <class 'str'>
202    <class 'str'>
203    <class 'str'>
204    <class 'str'>
205    <class 'str'>
Name: Rank, Length: 206, dtype: object

In [292]:
# Check Data Type in Rank
# See different data types in the same column. 

df['Rank'].apply(type).value_counts()

<class 'str'>    186
<class 'int'>     20
Name: Rank, dtype: int64

In [294]:
def clean_rank_data(x):
    """ If the value is a string, then remove currency symbol, delimiters, and N.A.
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', '').replace('N.A.','0'))
    return(x)

In [295]:
# Clean data and convert to float data type
df['Rank'] = df['Rank'].apply(clean_rank_data).astype('float')

In [296]:
# Check Data Type in Rank
# See float data types in the column. 

df['Rank'].apply(type).value_counts()

<class 'float'>    206
Name: Rank, dtype: int64

In [297]:
df.Rank

0          6.0
1        126.0
2        211.0
3         97.0
4        132.0
        ...   
201    13090.0
202    10271.0
203    25831.0
204    16544.0
205    11412.0
Name: Rank, Length: 206, dtype: float64

In [309]:
import altair as alt


brush = alt.selection_interval()

points = alt.Chart(df).mark_circle().encode(
    x='Est_Monthly_Sales:Q',
    y='Est_Monthly_Revenue:Q',
    color=alt.condition(brush, 'Category:N', alt.value('lightgray')), 
    tooltip=['Sellers', 'LQS', 'Reviews', 'Rank', 'Fees', 'Net', 'Est_Monthly_Sales','Est_Monthly_Revenue', 'Category', 'Product_Name']
).properties(width=400, height=200).add_selection(
    brush
)

bars = alt.Chart(df).mark_bar().encode(
    y='Category:N',
    color='Category:N',
    x='count(Category):Q'
).properties(width=400, height=200).transform_filter(
    brush
)


plot_product_scatterchart =  points & bars

plot_product_scatterchart

alt.VConcatChart(...)

In [318]:
plot_product_bar = alt.Chart(df).mark_bar().encode(
    x='LQS',
    y='Net:Q',
    color= 'Category:N', 
    tooltip=['Sellers', 'LQS', 'Reviews', 'Rank', 'Fees', 'Net', 'Est_Monthly_Sales','Est_Monthly_Revenue', 'Category', 'Product_Name']
).properties(width=400, height=200)

plot_product_bar

alt.Chart(...)

In [320]:
plot_product_line = alt.Chart(df).mark_line().encode(
    x='Rank',
    y='Reviews:Q',
    color= 'Category:N', 
    tooltip=['Sellers', 'LQS', 'Reviews', 'Rank', 'Fees', 'Net', 'Est_Monthly_Sales','Est_Monthly_Revenue', 'Category', 'Product_Name']
).properties(width=400, height=200)

plot_product_line

alt.Chart(...)

In [321]:
df['Date_First_Available'].apply(type)

0      <class 'str'>
1      <class 'str'>
2      <class 'str'>
3      <class 'str'>
4      <class 'str'>
           ...      
201    <class 'str'>
202    <class 'str'>
203    <class 'str'>
204    <class 'str'>
205    <class 'str'>
Name: Date_First_Available, Length: 206, dtype: object

In [338]:
alt.Chart(df).mark_bar().encode(
    x='year(Date_First_Available):T',
    y='Price',
    color='Category',    
    tooltip=['year(Date_First_Available)', 'Sellers', 'LQS', 'Reviews', 'Rank', 'Fees', 'Net', 'Est_Monthly_Sales','Est_Monthly_Revenue', 'Category', 'Product_Name']
).properties(
            height=180,
            width=500,
            ).interactive()

alt.Chart(...)

In [340]:
alt.Chart(df).mark_bar().encode(
    x='yearquartermonth(Date_First_Available):T',
    y='Price',
    color='Category',    
    tooltip=['yearquartermonth(Date_First_Available)', 'Sellers', 'LQS', 'Reviews', 'Rank', 'Fees', 'Net', 'Est_Monthly_Sales','Est_Monthly_Revenue', 'Category', 'Product_Name']
).properties(
            height=180,
            width=500,
            ).interactive()

alt.Chart(...)